In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# ====================
# Paso 1: Carga de Datos
# ====================
# Cargar los archivos
ruta_base = 'data/data_format1/data_format1/'
ruta_train = ruta_base + 'train_format1.csv'
ruta_test = ruta_base + 'test_format1.csv'
ruta_info_usuarios = ruta_base + 'user_info_format1.csv'
ruta_log_usuarios = ruta_base + 'user_log_format1.csv'

# Cargar datos
datos_train = pd.read_csv(ruta_train)
datos_test = pd.read_csv(ruta_test)
info_usuarios = pd.read_csv(ruta_info_usuarios)
log_usuarios = pd.read_csv(ruta_log_usuarios)

# Marcar datos de entrenamiento y prueba
datos_train['tipo'] = 'train'
datos_test['tipo'] = 'test'
datos_test = datos_test.drop('prob', axis=1)  # Eliminamos la columna de probabilidad en test

# Combinar datos de entrenamiento y prueba
conjunto_datos = pd.concat([datos_train, datos_test], ignore_index=True)

# ====================
# Paso 2: Preparación de Datos y Creación de Características Basadas en Double11 Day
# ====================
# Definir el Double11 Day como un día de referencia clave
DOUBLE11_DAY = 184  # Suponiendo que el día 184 en el dataset es el Double11

# Convertir 'time_stamp' a días transcurridos desde la fecha mínima
fecha_base = pd.to_datetime(log_usuarios['time_stamp'], format='%m%d').min()
log_usuarios['time_stamp'] = (pd.to_datetime(log_usuarios['time_stamp'], format='%m%d') - fecha_base).dt.days

# Calcular la recencia relativa a Double11 Day y agregar características
log_usuarios['double11_recencia'] = np.abs(log_usuarios['time_stamp'] - DOUBLE11_DAY)
user_agg = log_usuarios.groupby('user_id').agg(
    ultima_actividad_pre_double11=('time_stamp', lambda x: max(x[x < DOUBLE11_DAY], default=DOUBLE11_DAY)),
    ultima_actividad_post_double11=('time_stamp', lambda x: min(x[x > DOUBLE11_DAY], default=DOUBLE11_DAY)),
    interacciones_pre_double11=('time_stamp', lambda x: (x < DOUBLE11_DAY).sum()),
    interacciones_post_double11=('time_stamp', lambda x: (x > DOUBLE11_DAY).sum()),
    clicks_double11=('action_type', lambda x: ((x == 0) & (log_usuarios['time_stamp'] == DOUBLE11_DAY)).sum()),
    compras_double11=('action_type', lambda x: ((x == 2) & (log_usuarios['time_stamp'] == DOUBLE11_DAY)).sum())
).reset_index()

# Crear métricas adicionales basadas en la recencia
user_agg['dias_recencia_pre_double11'] = DOUBLE11_DAY - user_agg['ultima_actividad_pre_double11']
user_agg['dias_recencia_post_double11'] = user_agg['ultima_actividad_post_double11'] - DOUBLE11_DAY
user_agg['frecuencia_total'] = user_agg['interacciones_pre_double11'] + user_agg['interacciones_post_double11']
user_agg['recencia_total'] = user_agg['dias_recencia_pre_double11'] + user_agg['dias_recencia_post_double11']

# Integrar estas características al conjunto de datos principal
conjunto_datos = conjunto_datos.merge(user_agg, on='user_id', how='left')

# ====================
# Paso 3: Reducción de Dimensionalidad con PCA
# ====================
# Aplicar PCA manteniendo el 95% de la varianza
X = conjunto_datos[conjunto_datos['tipo'] == 'train'].drop(['label', 'user_id', 'tipo'], axis=1)
y = conjunto_datos[conjunto_datos['tipo'] == 'train']['label']

pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)

# ====================
# Paso 4: Optimización de Hiperparámetros y Ensemble de Modelos
# ====================
# Definir los parámetros para la búsqueda aleatoria
param_dist_xgb = {
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 300, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

param_dist_lgb = {
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 300, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Búsqueda aleatoria para XGBClassifier
xgb_random_search = RandomizedSearchCV(
    XGBClassifier(objective='binary:logistic', use_label_encoder=False, seed=42, eval_metric='auc'),
    param_distributions=param_dist_xgb,
    scoring='roc_auc',
    n_iter=10,
    cv=3,
    random_state=42
)
xgb_random_search.fit(X_reduced, y)
best_xgb = xgb_random_search.best_estimator_

# Búsqueda aleatoria para LGBMClassifier
lgb_random_search = RandomizedSearchCV(
    LGBMClassifier(random_state=42),
    param_distributions=param_dist_lgb,
    scoring='roc_auc',
    n_iter=10,
    cv=3,
    random_state=42
)
lgb_random_search.fit(X_reduced, y)
best_lgb = lgb_random_search.best_estimator_

# Crear el ensemble con los mejores modelos obtenidos
ensemble_model = VotingClassifier(
    estimators=[('xgb', best_xgb), ('lgb', best_lgb)],
    voting='soft'
)

# ====================
# Paso 5: Validación Cruzada Estratificada con Ensemble
# ====================
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
auc_scores = []

for train_index, valid_index in kf.split(X_reduced, y):
    X_train, X_valid = X_reduced[train_index], X_reduced[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    # Entrenar el ensemble
    ensemble_model.fit(X_train, y_train)
    
    # Calcular AUC en el conjunto de validación
    predictions_proba = ensemble_model.predict_proba(X_valid)[:, 1]
    auc_score = roc_auc_score(y_valid, predictions_proba)
    auc_scores.append(auc_score)

# Resultados finales
print("Promedio de AUC en validación cruzada tomando Double11 Day como referencia:", np.mean(auc_scores))

In [ ]:
import pickle

# Guardar el modelo ensemble entrenado
with open("modelo_ensemble_double11.pkl", "wb") as f:
    pickle.dump(ensemble_model, f)

In [ ]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

# Definir el tamaño de entrenamiento y la métrica de evaluación (AUC)
train_sizes, train_scores, valid_scores = learning_curve(
    ensemble_model, X_reduced, y, cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42),
    scoring='roc_auc', n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10)
)

# Calcular los promedios y desviaciones estándar de las puntuaciones
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
valid_mean = np.mean(valid_scores, axis=1)
valid_std = np.std(valid_scores, axis=1)

# Graficar las curvas de aprendizaje
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, 'o-', color="blue", label="AUC en Entrenamiento")
plt.plot(train_sizes, valid_mean, 'o-', color="red", label="AUC en Validación")

# Rellenar con la desviación estándar
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="blue", alpha=0.2)
plt.fill_between(train_sizes, valid_mean - valid_std, valid_mean + valid_std, color="red", alpha=0.2)

# Añadir etiquetas y título
plt.title("Curvas de Aprendizaje del Mejor Modelo")
plt.xlabel("Tamaño del Conjunto de Entrenamiento")
plt.ylabel("AUC")
plt.legend(loc="best")
plt.grid()
plt.show()
